In [2]:
import time
import os
import re
import numpy as np
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.nn.functional as F
import torchvision
from copy import deepcopy
import torchvision.transforms as transforms
from torch.utils.data.dataset import Subset
from PreResNet import ResNet18


import string
import matplotlib.pyplot as plt
from prettytable import PrettyTable
from IPython.display import Latex
from keras.datasets import cifar10
import tensorflow as tf

# gpu or cpu
device = torch.device("cuda")

Using TensorFlow backend.
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:526: FutureWarn

In [7]:
print(sum([[i]*2 for i in range(10)], []))

[0, 0, 1, 1, 2, 2, 3, 3, 4, 4, 5, 5, 6, 6, 7, 7, 8, 8, 9, 9]


In [12]:
a = [1,2,3]
print(a/np.sum(a))

[0.16666667 0.33333333 0.5       ]


In [26]:
print("Train teacher new")
table = PrettyTable([' ', 'lr=0.3', '0.2', '0.1', '0.01'])
test = ['w.o mixup'] 
for lr in ['0.300', '0.200', '0.100', '0.010']:
    acc = []
    for index in range(5):
        try:
            file = open('record/teacher_new/acc_lr_'+lr+'_'+str(index)+'.txt')               
            lines = file.readlines()
            test_acc = []
            for line in lines:
                new_line = re.split(':|,| |\n', line.strip())
                test_acc.append(eval(new_line[12]))
            acc.append(max(test_acc))
        except:
            pass
    if len(acc)>0:
        test.append(str(np.round(np.mean(acc),3)) + '(' + str(len(acc)) + ')')
    else:
        test.append('/')
table.add_row(test)

test = ['with mixup'] 
for lr in ['0.300', '0.200', '0.100', '0.010']:
    acc = []
    for index in range(20):
        try:
            file = open('record/teacher_mixup_new/acc_lr_'+lr+'_'+str(index)+'.txt')               
            lines = file.readlines()
            test_acc = []
            for line in lines:
                new_line = re.split(':|,| |\n', line.strip())
                test_acc.append(eval(new_line[12]))
            acc.append(max(test_acc))
        except:
            pass
    if len(acc)>0:
        test.append(str(np.round(np.mean(acc),3)) + '(' + str(len(acc)) + ')')
    else:
        test.append('/')
table.add_row(test)


test = ['weighted loss'] 
for lr in ['0.300', '0.200', '0.100', '0.010']:
    acc = []
    for index in range(5):
        try:
            file = open('record/teacher_weighted_loss_new2/acc_lr_'+lr+'_'+str(index)+'.txt')               
            lines = file.readlines()
            test_acc = []
            for line in lines:
                new_line = re.split(':|,| |\n', line.strip())
                test_acc.append(eval(new_line[12]))
            acc.append(max(test_acc))
        except:
            pass
    if len(acc)>0:
        test.append(str(np.round(np.mean(acc),3)) + '(' + str(len(acc)) + ')')
    else:
        test.append('/')
table.add_row(test)
print(table)

Train teacher new
+---------------+----------+----------+-----------+----------+
|               |  lr=0.3  |   0.2    |    0.1    |   0.01   |
+---------------+----------+----------+-----------+----------+
|   w.o mixup   | 0.435(5) | 0.662(3) |  0.666(5) | 0.663(3) |
|   with mixup  |    /     |  0.7(3)  | 0.698(16) | 0.698(3) |
| weighted loss |    /     |    /     |  0.596(4) | 0.648(1) |
+---------------+----------+----------+-----------+----------+


In [19]:
y = np.array([0]*5+[1]*5+[2]*5)
print(y)
index_list = []
for i in range(3):
    indices = np.where(y==i)[0]
    index_list.append(indices)
    print(indices)
print(index_list)

[0 0 0 0 0 1 1 1 1 1 2 2 2 2 2]
[0 1 2 3 4]
[5 6 7 8 9]
[10 11 12 13 14]
[array([0, 1, 2, 3, 4]), array([5, 6, 7, 8, 9]), array([10, 11, 12, 13, 14])]
